<a href="https://colab.research.google.com/github/aquilesRod/Bayesian-Neural-Network/blob/main/bayesian_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Tópicos em IA - Redes Bayesianas

## Sobre o projeto

> Implementação utilizando a biblioteca ***pgmpy*** (www.github.com/pgmpy)

> Sobre os dados:

>     Os dados foram obtidos pela National Center for Health Statistics (EUA) por meio de formulários da iniciativa NHANES,
>     estes foram coletados entre os anos de 2017 à 2018.

> Base de dados: https://wwwn.cdc.gov/nchs/nhanes/search/DataPage.aspx?Component=Questionnaire&Cycle=2017-2018

> Variáveis utilizadas:
>*   [A]: Falta de ar em escadas/rampas; ------------------------------> (Doc: CDQ_J, ColumnName: CDQ010)
>*   [B]: Resfriado nos últimos 30 dias; ---------------------------------> (Doc: HSQ_J, ColumnName: HSQ500)
>*   [C]: Dor de estômago/intestino nos últimos 30 dias; ------> (Doc: HSQ_J, ColumnName: HSQ510)
>*   [D]: Horas trabalhadas na última semana; ---------------------> (Doc: OCQ_J, ColumnName: OCQ180)

> Objetivo principal:

>*   Verificar se a quantidade de horas trabalhadas [A] é afetada pelas demais variáveis.

> Objetivos gerais:

>*   Estimar a estrutura do grafo gerado pelos dados, bem como adicionar probabilidades;
>*   Realizar inferências a partir do modelo gerado;

---

## Pré-processamento dos dados

###  Imports necessários



In [22]:
import pandas as pd
import numpy as np

### Leitura dos dados

In [23]:
data_frame_cardio_health = pd.read_csv("CDQ_J.csv")
data_frame_current_health_status = pd.read_csv("HSQ_J.csv")
data_frame_occupation = pd.read_csv("OCQ_J.csv")

### Extraindo as variáveis que serão utilizadas na rede

In [24]:
data_frame_shortness_of_breath = pd.DataFrame()
data_frame_cold = pd.DataFrame()
data_frame_stomach_illness = pd.DataFrame()
data_frame_hours_worked = pd.DataFrame()

data_frame_shortness_of_breath["SEQN"] = data_frame_cardio_health["SEQN"]
data_frame_shortness_of_breath["CDQ010"] = data_frame_cardio_health["CDQ010"]

data_frame_cold["SEQN"] = data_frame_current_health_status["SEQN"]
data_frame_cold["HSQ500"] = data_frame_current_health_status["HSQ500"]

data_frame_stomach_illness["SEQN"] = data_frame_current_health_status["SEQN"]
data_frame_stomach_illness["HSQ510"] = data_frame_current_health_status["HSQ510"]

data_frame_hours_worked["SEQN"] = data_frame_occupation["SEQN"]
data_frame_hours_worked["OCQ180"] = data_frame_occupation["OCQ180"]

data_frame_shortness_of_breath.dropna(inplace=True)
data_frame_cold.dropna(inplace=True)
data_frame_stomach_illness.dropna(inplace=True)
data_frame_hours_worked.dropna(inplace=True)

data_frame_auxiliary_for_merge_1 = pd.merge(data_frame_shortness_of_breath, data_frame_cold, on='SEQN', how='outer')
data_frame_auxiliary_for_merge_2 = pd.merge(data_frame_auxiliary_for_merge_1, data_frame_stomach_illness, on='SEQN', how='outer')
data_frame_merged = pd.merge(data_frame_auxiliary_for_merge_2, data_frame_hours_worked, on='SEQN', how='outer')

data_frame_merged.dropna(inplace=True)
data_frame_merged.head()

,SEQN,CDQ010,HSQ500,HSQ510,OCQ180
0,93705.0,2.0,2.0,2.0,30.0
3,93711.0,2.0,2.0,2.0,35.0
7,93716.0,2.0,2.0,2.0,40.0
9,93721.0,2.0,2.0,2.0,5.0
10,93722.0,2.0,2.0,2.0,40.0


### Alterando o nome das colunas para (A, B, C, D)

In [25]:
data_frame = data_frame_merged.rename(columns = {'CDQ010' : 'A',
                                                 'HSQ500' : 'B',
                                                 'HSQ510' : 'C',
                                                 'OCQ180' : 'D'})

data_frame.drop(['SEQN'], axis=1, inplace=True)
data_frame.head()

,A,B,C,D
0,2.0,2.0,2.0,30.0
3,2.0,2.0,2.0,35.0
7,2.0,2.0,2.0,40.0
9,2.0,2.0,2.0,5.0
10,2.0,2.0,2.0,40.0


#### Possíveis valores para cada variável:

>*   [A], [B] e [C] = (1- sim; 2- não; 9- não sabe);
>*   [D] = (6 a 78; 5; 80);

#### Lembrando que:

>*   [A] Falta de ar em escadas/rampas;
>*   [B] Resfriado nos últimos 30 dias;
>*   [C] Dor de estômago/intestino nos últimos 30 dias;
>*   [D] Horas trabalhadas na última semana;


### Reduzindo os valores da coluna [D] para diminuir a variância

In [26]:
data_frame.loc[data_frame['D'] < 30, "D"] = 0
data_frame.loc[(data_frame['D'] >= 30) & (data_frame['D'] < 60), "D"] = 1
data_frame.loc[data_frame['D'] >= 60, "D"] = 2

data_frame.head()

,A,B,C,D
0,2.0,2.0,2.0,1.0
3,2.0,2.0,2.0,1.0
7,2.0,2.0,2.0,1.0
9,2.0,2.0,2.0,0.0
10,2.0,2.0,2.0,1.0


## Modelo Bayesiano

### Instalando a biblioteca pgmpy

In [27]:
pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.8 MB/s eta 0:00:00


### Imports necessários

In [42]:
import pgmpy as pgm
from pgmpy.estimators import PC
from pgmpy.estimators import BayesianEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination

### Criando a estrutura da Rede Bayesiana

In [36]:
BNN_structure = PC(data=data_frame)
estimated_model = BNN_structure.estimate(variant='stable', max_cond_vars=2)

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pgmpy/estimators/CITests.py:547: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/usr/local/lib/python3.10/dist-packages/pgmpy/estimators/CITests.py:547: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):


### Visualização das dependências entre as variáveis (quem afeta quem?)

In [37]:
print(estimated_model.edges())

[('B', 'C'), ('A', 'C')]


#### Em outras palavras, é possível perceber que a probabilidade de uma pessoa ter contraído dor de estômago/intestino nos últimos 30 dias [C] é influenciada tanto pela falta de ar ao subir escadas [A] quanto pelo fato de ter contraído um resfriado nos últimos 30 dias [B].
#### No entanto, essa estimativa no mostra que nenhuma das variáveis [A], [B] ou [C] afetam a quantidade de horas trabalhadas por uma pessoa [D].

### Criação da Rede Bayesiana

In [40]:
BNN = BayesianNetwork(estimated_model.edges())
BNN.fit(data_frame)

### Exibição das tabelas das probabilidades condicionadas

In [41]:
BNN.get_cpds()

[<TabularCPD representing P(B:3) at 0x7e4b61a2bdc0>,
 <TabularCPD representing P(C:3 | A:3, B:3) at 0x7e4b61a29600>,
 <TabularCPD representing P(A:3) at 0x7e4b53d78040>]

### Realização das inferências

In [43]:
inferences = VariableElimination(BNN)

### Distribuição das probabilidades das variáveis

#### Distribuição de A

In [49]:
A_distribution = inferences.query(['A'])
print(A_distribution)

+--------+----------+
| A      |   phi(A) |
+========+==========+
| A(1.0) |   0.2699 |
+--------+----------+
| A(2.0) |   0.7282 |
+--------+----------+
| A(9.0) |   0.0019 |
+--------+----------+


##### Ou seja, esta tabela mostra qual é a probabilidade de uma pessoa sentir faltar de ar ao subir uma escada/rampa.

#### Distribuição de B

In [50]:
B_distribution = inferences.query(['B'])
print(B_distribution)

+--------+----------+
| B      |   phi(B) |
+========+==========+
| B(1.0) |   0.1631 |
+--------+----------+
| B(2.0) |   0.8350 |
+--------+----------+
| B(9.0) |   0.0019 |
+--------+----------+


##### Ou seja, esta tabela mostra qual é a probabilidade de uma pessoa ter contraído resfriado nos últimos 30 dias.

#### Distribuição de C

In [51]:
C_distribution = inferences.query(['C'])
print(C_distribution)

+--------+----------+
| C      |   phi(C) |
+========+==========+
| C(1.0) |   0.0556 |
+--------+----------+
| C(2.0) |   0.9435 |
+--------+----------+
| C(9.0) |   0.0010 |
+--------+----------+


##### Ou seja, esta tabela mostra qual é a probabilidade de uma pessoa ter sentido dor de estômago/intestino nos últimos 30 dias.

#### Distribuição de C, dado que A ocorreu

In [53]:
C_distribution_when_A_occurs = inferences.query(['C'], evidence={'A' : 1.00})
print(C_distribution_when_A_occurs)

+--------+----------+
| C      |   phi(C) |
+========+==========+
| C(1.0) |   0.0952 |
+--------+----------+
| C(2.0) |   0.9016 |
+--------+----------+
| C(9.0) |   0.0032 |
+--------+----------+


##### Ou seja, esta tabela mostra qual é a probabilidade de uma pessoa ter sentido dor de estômago/intestino nos últimos 30 dias, dado que essa mesma pessoa sente falta de ar ao subir uma escada/rampa.

##### Distribuição de C, dado que A e B ocorreram

In [55]:
C_distribution_when_A_and_B_occurs = inferences.query(['C'], evidence={'A' : 1.00, 'B' : 1.0})
print(C_distribution_when_A_and_B_occurs)

+--------+----------+
| C      |   phi(C) |
+========+==========+
| C(1.0) |   0.1609 |
+--------+----------+
| C(2.0) |   0.8391 |
+--------+----------+
| C(9.0) |   0.0000 |
+--------+----------+


##### Ou seja, esta tabela mostra qual é a probabilidade de uma pessoa ter sentido dor de estômago/intestino nos últimos 30 dias, dado que essa mesma pessoa sente falta de ar ao subir uma escada/rampa e contraiu um resfriado nos últimos 30 dias.

## Conclusões

##### Em suma, podemos perceber que a resposta para nosso questionamento detalhado no objetivo principal é: Não.
##### Não é possível determinar as horas trabalhadas de uma determinada pessoa a partir das variáveis trabalhadas neste projeto.
##### No entanto, pudemos notar que o evento "dor de estômago/intestino" possui sua probabilidade condicionada às chances dos eventos "Falta de ar em escadas/rampas" e "Resfriado nos últimos 30 dias" ocorrerem.
##### Além disso, observou-se que, mesmo diante da ausência de informações sobre a ocorrência de um evento específico, é possível  determinar a probabilidade de um evento condicionado a esse desconhecido, revelando, dessa forma, o notável poder das Redes Bayesianas, a flexibilidade.
##### Nas Redes Bayesianas, as probabilidades se ajustam conforme novas informações são fornecidas, diferentemente, por exemplo, das Redes Neurais Artificiais, que exigem um re-treinamento diante desse tipo de cenário.